In [29]:
  GNU nano 2.5.3                     File: q1_parquet.py                                                

from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType, TimestampTy$
from pyspark.sql.functions import col, hour, asc, udf, dayofmonth, desc,avg
import time
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext



t1 = time.time()


path_trip = "hdfs://master:9000/yellow_tripdata_1m.parquet"

# define a scructType schema for the dataframe


spark = SparkSession.builder.appName("q1_parquet").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)


trip_data = sqlContext.read.parquet(path_trip)


# set filters
trip_data = trip_data.filter(((col("trip_start_long") != col("trip_end_long")) & \
                             (col("trip_start_lat") != col("trip_end_lat"))) & \
                             (col("trip_start_time") < col("trip_end_time")) &\
                            (col("trip_start_lat") >= 40) & (col("trip_start_lat") <= 45) &\
                             (col("trip_end_lat") >= 40) & (col("trip_end_lat") <= 45) &\
                             (col("trip_start_long") >= -80) & (col("trip_start_long") <= -71) &\
                            (col("trip_end_long") >= -80) & (col("trip_end_long") <= -71))


# register table
trip_data.registerTempTable("trip_data")


trip_data = trip_data.withColumn("trip_start_time", hour(col("trip_start_time")))
#trip_data = trip_data.withColumn("trip_end_time", hour(col("trip_end_time")))

res = trip_data.groupBy('trip_start_time').agg({'trip_start_long': 'mean', 'trip_start_lat':'mean'}).so$

res.select("trip_start_time","avg(trip_start_long)","avg(trip_start_lat)").show()

t2 = time.time()

total_time = t2-t1

+---------------+-------------------+--------------------+
|trip_start_time|avg(trip_start_lat)|avg(trip_start_long)|
+---------------+-------------------+--------------------+
|              3| 40.627629672779754|  -73.77892133067635|
|              4|  40.56316505688749|  -73.65362635437539|
|              5|  40.63537364364952|  -73.77073302662073|
|              6|  40.61719897588094|  -73.72417994289533|
|              7| 40.638547884394185|  -73.76285024337749|
|              8|   40.6821769773265|   -73.8392740275867|
|              9| 40.684715533967264|  -73.84351477196546|
|             10|  40.65887830838559|  -73.79915788923404|
|             11|  40.68967540320586|   -73.8586070815505|
+---------------+-------------------+--------------------+



total time to run:  1.4332575798034668
